## Start by setting week number
#### Wait for ESPN data update until at least Wednesday morning, but run latest by Thursday night

https://www.espn.com/nfl/stats/team/_/stat/total/season/2021/seasontype/2

In [1]:
SET_WEEK = 16
week_name = 'week_' + str(SET_WEEK)
print(week_name)

week_16


In [2]:
from scrapers.scrape_2021 import get_2021_data
import pandas as pd
import numpy as np
import pickle

import warnings
warnings.filterwarnings('ignore')

### Models

In [3]:
rf = '../modeling/rf.pkl'
gbm = '../modeling/gbm.pkl'
lgb = '../modeling/lgb.pkl'
catboost = '../modeling/catboost.pkl'

with open(rf, 'rb') as file:
    RF = pickle.load(file)
    
with open(gbm, 'rb') as file:
    GBM = pickle.load(file)
    
with open(lgb, 'rb') as file:
    LGB = pickle.load(file)
    
with open(catboost, 'rb') as file:
    CATBOOST= pickle.load(file)

### 2021 data

In [4]:
%%time

X = get_2021_data()

# from boruta elimination method:
keep_cols = ['offense_points_per_game', 'offense_passing_AVG', 'offense_passing_YDS/G', 'offense_passing_RTG', 
             'offense_receiving_AVG', 'offense_rushing_YDS/G', 'defense_points_per_game', 'defense_passing_AVG', 
             'defense_receiving_AVG', 'defense_rushing_YDS/G', 'defense_passing_SYL_per_game', 
             'offense_downs_Third Downs_ATT_per_game', 'defense_downs_Third Downs_ATT_per_game', 
             'defense_downs_Fourth Downs_ATT_per_game', 'defense_pass_to_rush_ratio', 
             'defense_downs_First Downs_penalty_ratio', 'offense_passing_TD_per_game', 'offense_pass_TD_to_INT']

X = X[['Team', 'games_played', 'season'] + keep_cols].copy()

X.head(2)

Wall time: 40.5 s


,Team,games_played,season,offense_points_per_game,offense_passing_AVG,offense_passing_YDS/G,offense_passing_RTG,offense_receiving_AVG,offense_rushing_YDS/G,defense_points_per_game,...,defense_receiving_AVG,defense_rushing_YDS/G,defense_passing_SYL_per_game,offense_downs_Third Downs_ATT_per_game,defense_downs_Third Downs_ATT_per_game,defense_downs_Fourth Downs_ATT_per_game,defense_pass_to_rush_ratio,defense_downs_First Downs_penalty_ratio,offense_passing_TD_per_game,offense_pass_TD_to_INT
0,Dallas Cowboys,15,2021,30.5,7.6,282.7,101.2,11.1,126.9,20.5,...,12.2,109.5,17.400000,13.600000,12.6,1.4,1.480769,0.117021,2.133333,2.909091
1,Tampa Bay Buccaneers,15,2021,29.5,7.2,300.0,100.0,10.8,101.7,20.8,...,9.8,87.5,17.533333,13.133333,13.4,1.4,1.923567,0.111842,2.466667,3.363636


In [5]:
X.fillna(0, inplace = True)
X.replace([np.inf, -np.inf], 0, inplace=True)

In [6]:
X['games_played'].unique().tolist()

[15]

In [7]:
# X[X['games_played'] == 8]

In [8]:
predictions = X[['Team', 'season']].copy()
predictions['week_marker'] = week_name

predictions['pred_rf'] = RF.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_gbm'] = GBM.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_lgb'] = LGB.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_catboost'] = CATBOOST.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_mean'] = (predictions['pred_rf'] + predictions['pred_gbm'] + predictions['pred_lgb'] + predictions['pred_catboost']) / 4

predictions.sort_values('pred_mean', ascending = False, inplace = True)
predictions.head(3)

,Team,season,week_marker,pred_rf,pred_gbm,pred_lgb,pred_catboost,pred_mean
6,Los Angeles Rams,2021,week_16,0.226068,0.164166,0.242199,0.258026,0.222615
1,Tampa Bay Buccaneers,2021,week_16,0.239440,0.164330,0.147299,0.228839,0.194977
5,Buffalo Bills,2021,week_16,0.227910,0.119966,0.143688,0.226860,0.179606


In [9]:
predictions.to_csv('weekly_runs/' + week_name + '.csv', index = False)

### Check weekly run

In [10]:
predictions

,Team,season,week_marker,pred_rf,pred_gbm,pred_lgb,pred_catboost,pred_mean
6,Los Angeles Rams,2021,week_16,0.226068,0.164166,0.242199,0.258026,0.222615
1,Tampa Bay Buccaneers,2021,week_16,0.239440,0.164330,0.147299,0.228839,0.194977
5,Buffalo Bills,2021,week_16,0.227910,0.119966,0.143688,0.226860,0.179606
0,Dallas Cowboys,2021,week_16,0.211819,0.141237,0.107692,0.200863,0.165403
2,Kansas City Chiefs,2021,week_16,0.175272,0.088869,0.097722,0.206076,0.141985
9,San Francisco 49ers,2021,week_16,0.142922,0.073787,0.096187,0.232881,0.136444
11,Cincinnati Bengals,2021,week_16,0.161259,0.084849,0.098904,0.188448,0.133365
7,Arizona Cardinals,2021,week_16,0.083187,0.051274,0.088053,0.190765,0.103320
13,Indianapolis Colts,2021,week_16,0.111151,0.058571,0.068160,0.167435,0.101329
14,Green Bay Packers,2021,week_16,0.092286,0.044983,0.061928,0.199170,0.099592
